# Praktikum Pengantar Pembelajaran Mesin


---
## Bab 4. Klasifikasi Naive Bayes


### 1) Import Data

Unduh dataset yang akan digunakan pada praktikum kali ini. Anda dapat menggunakan aplikasi wget untuk mendowload dataset dan menyimpannya dalam Google Colab. Jalankan cell di bawah ini untuk mengunduh dataset

In [237]:
!wget https://gist.githubusercontent.com/netj/8836201/raw/6f9306ad21398ea43cba4f7d537619d0e07d5ae3/iris.csv

'wget' is not recognized as an internal or external command,
operable program or batch file.


Setelah dataset berhasil diunduh, langkah berikutnya adalah membaca dataset dengan memanfaatkan fungsi **readcsv** dari library pandas. Lakukan pembacaan berkas csv ke dalam dataframe dengan nama **data** menggunakan fungsi **readcsv**. Jangan lupa untuk melakukan import library pandas terlebih dahulu


In [238]:
import pandas as pd

df = pd.read_csv('iris.csv')



Cek isi dataset Anda dengan menggunakan perintah **head()**

In [239]:
df.head()

,sepal.length,sepal.width,petal.length,petal.width,variety
0,5.1,3.5,1.4,0.2,Setosa
1,4.9,3.0,1.4,0.2,Setosa
2,4.7,3.2,1.3,0.2,Setosa
3,4.6,3.1,1.5,0.2,Setosa
4,5.0,3.6,1.4,0.2,Setosa


## 2) Membagi data menjadi data latih dan data uji

Metode pembelajaran mesin memerlukan dua jenis data :


1.   Data latih : Digunakan untuk proses training metode klasifikasi
2.   Data uji : Digunakan untuk proses evaluasi metode klasifikasi

Data uji dan data latih perlu dibuat terpisah (mutualy exclusive) agar hasil evaluasi lebih akurat.

Data uji dan data latih dapat dibuat dengan cara membagi dataset dengan rasio tertentu, misalnya 80% data latih dan 20% data uji.

Library Scikit-learn memiliki fungsi [train_test_split](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html) pada modul **model_selection** untuk membagi dataset menjadi data latih dan data uji. Bagilah dataset anda menjadi dua, yaitu **data_latih** dan **data_uji**.


In [240]:
from sklearn.model_selection import train_test_split

df_train, df_test = train_test_split(df, test_size=0.2)

Tampilkan banyaknya data pada **data_latih** dan **data_uji**. Seharusnya **data_latih** terdiri dari 120 data, dan **data_uji** terdiri dari 30 data

In [241]:
print(df_train.shape[0])
print(df_test.shape[0])

120
30


Pisahkan label/kategori dari data uji menjadi variabel tersendiri. Beri nama variabelnya  **label_uji**

In [242]:
label_train = df_train.pop('variety')
label_test = df_test.pop('variety')

In [243]:
label_train.head()

109     Virginica
77     Versicolor
117     Virginica
104     Virginica
118     Virginica
Name: variety, dtype: object

## 3) Menghitung Prior

Tahapan pertama pada algoritma Naive Bayes adalah perhitungan prior. Prior suatu kelas merupakan peluang awal munculnya kelas tersebut. Atau dengan kata lain, prior merupakan frekuensi relatif dari suatu kelas terhadap keseluruhan data. 

Buatlah fungsi bernama **hitung_prior** yang berfungsi menghitung nilai prior dari masing-masing kelas yang terdapat pada data latih. Fungsi ini menerima input berupa list kelas dari data latih.

In [244]:
from collections import Counter

def count_prior(list_label):
  n_data = len(list_label)
  prior = Counter(list_label)
  for key in prior.keys():
    prior[key] = prior[key] / n_data
  return prior

Ujilah fungsi **hitung_prior** menggunakan kelas dari variabel **data_latih**. Amati hasilnya

In [245]:
prior = count_prior(label_train)
print(prior)

Counter({'Setosa': 0.35833333333333334, 'Virginica': 0.325, 'Versicolor': 0.31666666666666665})


## 4) Menghitung *likelihood*

Praktikum kali ini mengimplementasikan metode Naive Bayes menggunakan datset iris. Semua fitur pada dataset iris memiliki tipe numerik. Dengan demikian, perhitungan *likelihood* dilakukan menggunakan Gaussian Model.

Tahapan yang perlu dilakukan sebelum perhitungan *likelihood* adalah menghitung rata-rata dan deviasi standar masing-masing fitur per kelas. Buatlah fungsi **hitung_rata2_std_kelas** yang berfungsi menghitung rata-rata dan standar deviasi per fitur dan kelas

In [246]:
def count_mean_std_label(input_data):
  list_columns = input_data.columns[:-1]
  class_columns_name = input_data.columns[-1]
  list_class = set(input_data[class_columns_name])
  mean = {}
  std = {}
  
  for column in list_columns:
    for a_class in list_class:
      mean[(a_class, column)] = input_data.loc[input_data[class_columns_name] == a_class][column].mean()
      std[(a_class, column)] = input_data.loc[input_data[class_columns_name] == a_class][column].std()
      
  return (mean, std)

Buatlah fungsi **hitung_likelihood_gaussian** yang bertujuan menghitung nilai likelihood suatu nilai terhadap rata-rata dan deviasi standar tertentu.

In [247]:
import math 

def count_likelyhood_gaussian(data, mean, std):
  try:
    result = (1/math.sqrt(2*math.pi*(std**2)))*math.exp((-1*((data-mean)**2))/(2*(std**2)))
    return result
  except ZeroDivisionError:
    return print('Error: Division by zero')

## 5) Proses *training*
Proses training pada Naive Bayes dengan model Gaussian dilakukan untuk menghitung prior serta  rata-rata dan deviasi standar pada masing-msaing fitur dan kelas. 

In [248]:
def training_naive_bayes_gaussian(data_train):
  class_columns_name = data_train.columns[-1]
  prior = count_prior(data_train[class_columns_name])
  (mean, std) = count_mean_std_label(data_train)
  list_class = set(data_train[class_columns_name])
  list_columns = data_train.columns[:-1]
  model = {}
  model['prior'] = prior
  model['mean'] = mean
  model['std'] = std
  model['list_class'] = list_class
  model['list_columns'] = list_columns
  return model

Lakukanlah proses training dan simpan hasil training pada variabel **model**

In [249]:
model = training_naive_bayes_gaussian(df)

## 6) Proses *testing*
Proses testing dilakukan dengan menghitung nilai posterior dari data uji berdasarkan nilai model yang diperoleh saat training. Nilai posterior diperoleh dari perkalian prior dengan *likelihood* masing-masing fiturnya. Setelah itu, penentuan kelas data uji dilakukan berdasarkan nilai posterior terbesar

In [250]:
def testing_naive_bayes_gaussian(model, data_test):
  prior = model['prior']
  mean = model['mean']
  std = model['std']
  list_class = model['list_class']
  list_columns = model['list_columns']
  posterior = dict.fromkeys(list_class, 1)
  for a_class in list_class:
    for column in list_columns:
      posterior[a_class] = posterior[a_class] * count_likelyhood_gaussian(data_test[column], mean[(a_class, column)], std[(a_class, column)])
    posterior[a_class] = posterior[a_class] * prior[a_class]
  predict_class = max(posterior, key=posterior.get)
  return predict_class
  

Ujilah fungsi **testing_naive_bayes_gaussian** menggunakan data uji pertama. Bandingkan hasilnya dengan nilai label sebenarnya.

In [251]:
index_test = 0
prediction = testing_naive_bayes_gaussian(model, df_test.iloc[index_test])
print(prediction)
print(label_test.iloc[index_test])

Versicolor
Versicolor


Lakukan pengujian untuk semua data uji. Selanjutnya bandingkan dengan label sebenarnya untuk menghitung berapa banyak data uji yang berhasil diprediksi dengan benar

In [252]:
total_prediction = []
for index in range (df_test.shape[0]):
  total_prediction.append(testing_naive_bayes_gaussian(model, df_test.iloc[index]))

print("Total prediksi benar: ",sum(total_prediction==label_test))

Total prediksi benar:  29


## TUGAS
Pada tugas kali ini Anda akan menggunakan dataset **[Car Evaluation Dataset](https://archive.ics.uci.edu/ml/datasets/Car+Evaluation)** yang telah dimodifikasi. Fitur-fitur yang ada bertipe kategori sebagai berikut:
1. **buying**. Merupakan harga beli mobil. Nilai : vhigh, high, med, low
2. **maint**. Menandakan biaya perawatan mobil. Nilai: vhigh, high, med, low
3. **lug_boot**. Menandakan ukuran bagasi. Nilai: small, med, big
4. **safety**. Menandakan skor keamanan mobil. Nilai:  low, med, high
5. **class**. Merupakan kelas data. Nilai: unacc, acc, good, vgood

Download dataset dari https://dataset-ppm.s3.amazonaws.com/car_sample.csv dan letakkan dalam variabel **data_tugas**.

In [253]:
!wget https://dataset-ppm.s3.amazonaws.com/car_sample.csv

'wget' is not recognized as an internal or external command,
operable program or batch file.


In [254]:
import pandas as pd
df = pd.read_csv('car_evaluation.csv')
df.head()

,buying,maint,lug_boot,safety,class
0,vhigh,vhigh,small,low,unacc
1,vhigh,vhigh,small,med,unacc
2,vhigh,vhigh,small,high,unacc
3,vhigh,vhigh,med,low,unacc
4,vhigh,vhigh,med,med,unacc


Bagi data menjadi data latih dan uji dengan nama **tugas_latih** dan **tugas_uji** dengan rasio 70%:30%

In [255]:
from sklearn.model_selection import train_test_split

df_train, df_test = train_test_split(df, test_size=0.3)

Pisahkan label/kategori dari data uji menjadi variabel tersendiri. Beri nama variabelnya **tugas_label_uji**

In [256]:
label_train = df_train.pop('class')
label_test = df_test.pop('class')

Hitunglah nilai prior dari data latih dan simpan nilainya pada variabel bernama **prior_tugas**. Anda dapat menggunakan fungsi **hitung_prior** yang telah Anda buat.

In [257]:
prior_task = count_prior(label_train)
print(prior_task)

Counter({'unacc': 0.7005789909015715, 'acc': 0.22663358147229115, 'vgood': 0.036393713813068655, 'good': 0.036393713813068655})


Buatlah fungsi bernama **hitung_likelihood_multinomial** dengan parameter input berupa data latih. Fungsi ini bertujuan menghitung nilai likelihood pada masing-masing kolom pada data latih. Operasi-operasi yang perlu dilakukan pada fungsi ini adalah:


1.   Mendapatkan daftar kolom data
2.   Mendapatkan nama kolom kelas pada data (kolom terakhir pada data)
3.   Mendapatkan nama-nama kelas pada data (unik).
4.   Menghitung likelihood pada masing-masing kolom, nilai kolom, dan nilai kelas

Sebagai saran, Anda dapat menggunakan struktur data dictionary untuk menyimpan nilai likelihood. Key pada dictionary tersebut adalah tuple(nama kolom, nilai pada kolom, nama kelas). Petunjuk penggunaan dictionary dengan key berupa tuple dapat dibaca [di sini](https://stackoverflow.com/questions/1938614/in-what-case-would-i-use-a-tuple-as-a-dictionary-key).



In [258]:
def hitung_likelihood_multinomial(data_latih):
    list_columns = data_latih.columns[:-1]
    class_column_name = data_latih.columns[-1]
    list_class = data_latih[class_column_name].unique()

    likelihood = {}
    for kolom in list_columns:
        for nilai_kolom in data_latih[kolom].unique():
            for a_class in list_class:
                count_nom = data_latih[(data_latih[kolom] == nilai_kolom) & (data_latih[class_column_name] == a_class)].shape[0]
                count_denom = data_latih[data_latih[class_column_name] == a_class].shape[0]
                likelihood[(kolom, nilai_kolom, a_class)] = count_nom / count_denom if count_denom != 0 else 0

    return likelihood


Ujilah fungsi **hitung_likelihood_multinomial** dengan parameter input berupa variabel **tugas_latih**

In [259]:
likelihood = hitung_likelihood_multinomial(df)
print(likelihood)

{('buying', 'vhigh', 'unacc'): 0.2975206611570248, ('buying', 'vhigh', 'acc'): 0.1875, ('buying', 'vhigh', 'vgood'): 0.0, ('buying', 'vhigh', 'good'): 0.0, ('buying', 'high', 'unacc'): 0.26776859504132233, ('buying', 'high', 'acc'): 0.28125, ('buying', 'high', 'vgood'): 0.0, ('buying', 'high', 'good'): 0.0, ('buying', 'med', 'unacc'): 0.22148760330578512, ('buying', 'med', 'acc'): 0.2994791666666667, ('buying', 'med', 'vgood'): 0.4, ('buying', 'med', 'good'): 0.3333333333333333, ('buying', 'low', 'unacc'): 0.21322314049586777, ('buying', 'low', 'acc'): 0.23177083333333334, ('buying', 'low', 'vgood'): 0.6, ('buying', 'low', 'good'): 0.6666666666666666, ('maint', 'vhigh', 'unacc'): 0.2975206611570248, ('maint', 'vhigh', 'acc'): 0.1875, ('maint', 'vhigh', 'vgood'): 0.0, ('maint', 'vhigh', 'good'): 0.0, ('maint', 'high', 'unacc'): 0.25950413223140495, ('maint', 'high', 'acc'): 0.2734375, ('maint', 'high', 'vgood'): 0.2, ('maint', 'high', 'good'): 0.0, ('maint', 'med', 'unacc'): 0.221487603

Buatlah fungsi bernama **training_naive_bayes_multinomial** dengan input berupa data latih. Fungsi ini bertujuan membentuk model yang berisi:


1.   Nilai prior semua kelas
2.   Nilai likelihood semua kolom, nilai kolom, dan kelas
3.   Daftar kelas pada data latih (unik)
4.   Daftar nama kolom pada data latih

Return value dari fungsi ini adalah **model**



In [260]:
def training_naive_bayes_multinomial(data_latih):
    likelihood = hitung_likelihood_multinomial(data_latih)
    kelas_data = data_latih[data_latih.columns[-1]].unique()
    kolom_data = list(data_latih.columns)

    model = {
        "prior": prior_task,
        "likelihood": likelihood,
        "class_data": kelas_data,
        "column_data": kolom_data
    }

    return model


Jalankan fungsi **training_naive_bayes_multinomial** dan simpan hasilnya dalam variabel **model_tugas**

In [261]:
model = training_naive_bayes_multinomial(df)

Buatlah fungsi **testing_naive_bayes_multinomial** untuk mendapatkan kelas dari sebuah data uji. Langkah yang dilakukan hampir sama dengan Gaussian, yaitu mengalikan likelihood semua kolom dengan posterior.

In [262]:
def testing_naive_bayes_multinomial(model,data_uji):
  prior = model["prior"]
  likelihood = model["likelihood"]
  kelas_data = model["class_data"]

  posterior = {}
  for kelas in kelas_data:
    posterior[kelas] = prior[kelas]
    for kolom in model["column_data"]:
      if kolom in data_uji:
        nilai_kolom = data_uji[kolom]
        posterior[kelas] *= likelihood[(kolom, nilai_kolom, kelas)]
  kelas_uji = max(posterior, key=posterior.get)
  
  return kelas_uji

Lakukan pengujian menggunakan sebuah data uji

In [263]:
idx = 10
test = df_test.iloc[idx]
class_result = testing_naive_bayes_multinomial(model,test)
print(class_result)
print(label_test.iloc[idx])

unacc
unacc


Lakukan pengujian terhadap semua data uji.

In [264]:
total_prediction =[]
for indeks in range(df_test.shape[0]):
  total_prediction.append(testing_naive_bayes_multinomial(model,df_test.iloc[indeks]))
  
print("Total prediksi benar: ",sum(total_prediction==label_test))

Total prediksi benar:  383


In [265]:
len(label_test)

519

Kesimpulan

Metode Naive Bayes adalah salah satu algoritma klasifikasi yang sederhana namun efektif dalam pembelajaran mesin. Meskipun asumsi dasarnya tentang independensi fitur sering kali tidak realistis dalam konteks dunia nyata, Naive Bayes masih sering memberikan hasil yang baik dalam praktiknya.

Menggunakan Teorema Bayes untuk menghitung probabilitas kelas yang mungkin berdasarkan pada probabilitas prior, likelihood, dan probabilitas evidence.
Meskipun asumsi bahwa setiap fitur adalah independen satu sama lain ketika diberikan kelas target (Naive Bayes assumption) sering kali tidak realistis, namun Naive Bayes masih sering memberikan hasil yang baik dalam praktiknya.
Cocok digunakan untuk berbagai tugas klasifikasi, terutama dalam kasus-kasus dengan dataset besar.

Naive Bayes dapat diimplementasikan dengan berbagai distribusi conditional probability, termasuk Bernoulli, Multinomial, dan Gaussian.
Bernoulli Naive Bayes cocok digunakan untuk data biner, Multinomial Naive Bayes cocok digunakan untuk data frekuensi, dan Gaussian Naive Bayes cocok digunakan untuk data kontinu.

Pemilihan jenis Naive Bayes yang tepat tergantung pada jenis data yang dimiliki. Setiap distribusi conditional probability mengasumsikan sesuatu tentang sifat data dan dapat memberikan hasil yang optimal untuk jenis data yang sesuai.
Dalam prakteknya, Naive Bayes adalah pilihan yang populer karena sederhana, cepat, dan seringkali cukup efektif, terutama dalam kasus-kasus di mana asumsi dasarnya dapat diterima atau dalam skenario dengan dataset besar di mana kecepatan komputasi penting. Dengan pemahaman yang baik tentang asumsi dan implementasi berbagai jenis Naive Bayes, kita dapat memilih dan menyesuaikan model yang sesuai dengan kebutuhan analisis data kita.